# generate_query

In [1]:
import mlrun

## 1. Configuration

In [2]:
project_name = "debug"
image = "dragon:30500/mlrun/mlrun:1.9.1"
sample_file = "sample.sql"

In [3]:
# get the debug project
project = mlrun.get_or_create_project(
    name=project_name,
    context="./",
    user_project=True
)

> 2025-08-21 13:49:35,718 [info] Created and saved project: {"context":"./","from_template":null,"name":"debug-johannes","overwrite":false,"save":true}
> 2025-08-21 13:49:35,718 [info] Project created successfully: {"project_name":"debug-johannes","stored_in_db":true}


In [4]:
# log the artifact
project.log_artifact(
    item="sample-sql",
    local_path=sample_file,
    format="sql",
    labels={ "Purpose": "debug" }
)

In [5]:
sql_artifact = project.get_artifact("sample-sql")

## 2. Run Locally

In [6]:
function = mlrun.code_to_function(
    name="generate_query",
    filename="generate_query.py",
    kind="job",
    image=image,
    handler="generate_query",
    requirements_file="requirements.txt"
)

In [7]:
result = function.run(
    inputs= {
        "input_file": sql_artifact.uri
    },
    params={
        "replacements": {
            "catalog": "iceberg",
            "schema": "lakehouse",
            "source_table": "taxi_trips",
            "filter_column": "trip_start_timestamp",
            "filter_start_value": "2023-05-01"
        }
    },
    local=True
)

> 2025-08-21 13:49:35,834 [warning] It is recommended to use k8s secret (specify secret_name), specifying aws_access_key/aws_secret_key directly is unsafe.
> 2025-08-21 13:49:35,839 [info] Storing function: {"db":"http://dragon.local:30070","name":"generate-query-generate-query","uid":"182394fed9ae4f008ef5beef7e345901"}
> 2025-08-21 13:49:35,924 [info] downloading s3://mlrun/projects/debug-johannes/artifacts/sample-sql.sql to local temp file
> 2025-08-21 13:49:35,997 [info] Loading SQL template from artifact
> 2025-08-21 13:49:35,998 [info] Performing replacements in the SQL template


project,uid,iter,start,end,state,kind,name,labels,inputs,parameters,results
debug-johannes,...345901,0,Aug 21 11:49:35,NaT,completed,run,generate-query-generate-query,v3io_user=johanneskind=localowner=johanneshost=dragon,input_file,"replacements={'catalog': 'iceberg', 'schema': 'lakehouse', 'source_table': 'taxi_trips', 'filter_column': 'trip_start_timestamp', 'filter_start_value': '2023-05-01'}","sql=SELECT\n *\nFROM\n ""iceberg"".""lakehouse"".""taxi_trips""\nWHERE\n CAST(trip_start_timestamp AS DATE) BETWEEN\n (CAST('2023-05-01' AS DATE) - INTERVAL '3' MONTH) AND\n (CAST('2023-05-01' AS DATE) - INTERVAL '2' MONTH)"


> 2025-08-21 13:49:36,039 [info] Run execution finished: {"name":"generate-query-generate-query","status":"completed"}


In [8]:
print(result.outputs["sql"])

SELECT
    *
FROM
    "iceberg"."lakehouse"."taxi_trips"
WHERE
    CAST(trip_start_timestamp AS DATE) BETWEEN
        (CAST('2023-05-01' AS DATE) - INTERVAL '3' MONTH) AND
        (CAST('2023-05-01' AS DATE) - INTERVAL '2' MONTH)
